<a href="https://colab.research.google.com/github/oldairjsilva/IGTI/blob/main/coleta_dados_web_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparando Ambiente
Precisamos instalar a versão 3.141.0 selenium em nossa virtualenv

In [ ]:
!pip install selenium==3.141.0

In [ ]:
import selenium

print("Versão utilizada", selenium.__version__)

Versão utilizada 3.141.0


In [ ]:
# importando o webdriver
from selenium import webdriver

# Atribuindo Valores anotados a variáveis

In [ ]:
# Dados Site

site = 'https://veiculos.fipe.org.br/'

# Tipo consulta
xpath_tipo_consulta = '//*[@id="front"]/div[1]/div[1]/ul/li[1]/a'

# Caixas de seleção
id_marca_carro = 'selectMarcacarro_chosen'
id_modelo_carro = 'selectAnoModelocarro_chosen'
id_ano_carro = 'selectAnocarro_chosen'
id_tabela_referencia = 'selectTabelaReferenciacarro_chosen'

# Botoes
id_botao_limpar = 'buttonLimparPesquisarcarro'
id_botao_pesquisa = 'buttonPesquisarcarro'

# Resultado da pesquisa
id_resultado_filtros = 'resultadoConsultacarroFiltros'

# Informacoes do carro
ano_veiculo = '2013'
modelo_veiculo = 'Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p'
marca_veiculo = 'Fiat'

# Caminho arquivo web driver
WEBDRIVER = 'C:\\Projetos\\Driver Selenium\\chromedriver.exe'

## Input de informações

In [ ]:
def x_path_input(id_elemento):
    return '//*[@id="{}"]/div/div/input'.format(id_elemento)

## Tempo de resposta do navegador:
Como estamos simulando a ação de um usuário a com o selenium precisamos esperar que as informações sejam redenrizadas no navegador, por esse motivo, a cada execução precisamos esperar que o html seja atualizado.

In [ ]:
# Metodo de esperar 
import random
import time

def esperar(segundos = None):
    """Caso nenhum valor seja atribuído, será atribuído um tempo randomico entre 2 e 8 
    (Feito desta forma para simular a ação de um usuário"""
    if not segundos:
        segundos = random.randrange(2, 8)
    time.sleep(segundos)

In [ ]:
# Abrindo o navegador, utilizando o drive do chrome
driver = webdriver.Chrome(WEBDRIVER)
driver.get(site)
esperar(15) # espera 15 segundos para executar outra interação

In [ ]:
# Vamos clicar no tipo de consulta escolhido
# Metodo find_element_by_xpath encontra um elemento atraves do xpath
tipo_consulta = driver.find_element_by_xpath(xpath_tipo_consulta)
tipo_consulta.click()

### Agora vamos inserir o período de referência

In [ ]:
# Metodo find_element_by_id encontra um elemento atraves do id
pesquisa = driver.find_element_by_id(id_tabela_referencia)
pesquisa.click()

In [ ]:
# retornamos o elemento de input e buscamos ele
xpath = x_path_input(id_tabela_referencia)
caixa = driver.find_element_by_xpath(xpath)

In [ ]:
from selenium.webdriver.common.keys import Keys

# Utilizamos o metodo send_key para inserir o valor desejado
# Importamos o metodo keys para simular as ações down e return do teclado
caixa.send_keys('fevereiro/2022' + Keys.DOWN + Keys.RETURN)

### Criando preencher de item
Como este procedimento será repetido em todas as caixas de seleção, podemos criar uma função

In [ ]:
def preenche_item(id_elemento, texto_pesquisa):
    pesquisa = driver.find_element_by_id(id_elemento)
    pesquisa.click()
    xpath = x_path_input(id_elemento)
    caixa = driver.find_element_by_xpath(xpath)
    caixa.send_keys(texto_pesquisa + Keys.DOWN + Keys.RETURN)
    pesquisa.is_selecteds = False # retirar seleção
    caixa.is_selecteds = False # retirar seleção

In [ ]:
# Resumindo
esperar()
preenche_item(id_tabela_referencia, 'fevereiro/2022')

In [ ]:
# Preencher marca
esperar()
preenche_item(id_marca_carro, marca_veiculo)

In [ ]:
# Preencher modelo
esperar()
preenche_item(id_modelo_carro, modelo_veiculo)

In [ ]:
# Preencher ano
esperar()
preenche_item(id_ano_carro, ano_veiculo)

In [ ]:
# Clicando em pesquisar
esperar()
buscar = driver.find_element_by_id(id_botao_pesquisa)
buscar.click()

In [ ]:
from bs4 import BeautifulSoup

# copiando resultados 

table = driver.find_element_by_id(id_resultado_filtros)
html = table.get_attribute('innerHTML')
parser = BeautifulSoup(html, 'html.parser')
table = parser.select_one('tbody')
table

<tbody>
<tr>
<td class="noborder"><p>Mês de referência:</p></td>
<td><p>fevereiro de 2022 </p></td>
</tr>
<tr>
<td class="noborder"><p>Código Fipe:</p></td>
<td><p>001314-5</p></td>
</tr>
<tr>
<td class="noborder"><p>Marca:</p></td>
<td><p>Fiat</p></td>
</tr>
<tr>
<td class="noborder"><p>Modelo:</p></td>
<td><p>Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p</p></td>
</tr>
<tr>
<td class="noborder"><p>Ano Modelo:</p></td>
<td><p>2013 Gasolina</p></td>
</tr>
<tr>
<td class="noborder"><p>Autenticação</p></td>
<td><p>rgy5qt8drlnc</p></td>
</tr>
<tr>
<td class="noborder"><p>Data da consulta</p></td>
<td><p>domingo, 27 de março de 2022 15:54</p></td>
</tr>
<tr class="last">
<td class="noborder"><p>Preço Médio</p></td>
<td><p>R$ 34.767,00</p></td>
</tr>
</tbody>

#### Observe que os valores que procuramos estão entre as tags td e p
Montaremos uma expressão regular para buscar esses elementos

In [ ]:
import re
"""
Caracteres inicial de busca
<td><p>

Caracteres finais de busca
</p></td>

# Por fim solicitamos qualquer elemento entre esses dois grupos de caracteres
"""

regex_dados = re.compile('(<td><p>)(.*?)(</p></td>)')

In [ ]:
# Agora informamos onde queremos buscar os dados
dados = regex_dados.findall(str(table))
dados

[('<td><p>', 'fevereiro de 2022 ', '</p></td>'),
 ('<td><p>', '001314-5', '</p></td>'),
 ('<td><p>', 'Fiat', '</p></td>'),
 ('<td><p>', 'Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p', '</p></td>'),
 ('<td><p>', '2013 Gasolina', '</p></td>'),
 ('<td><p>', 'rgy5qt8drlnc', '</p></td>'),
 ('<td><p>', 'domingo, 27 de março de 2022 15:54', '</p></td>'),
 ('<td><p>', 'R$ 34.767,00', '</p></td>')]

In [ ]:
# Tornando mais legível
for i, dado in enumerate(dados):
    dados[i] = dado[1]
dados

['fevereiro de 2022 ',
 '001314-5',
 'Fiat',
 'Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p',
 '2013 Gasolina',
 'rgy5qt8drlnc',
 'domingo, 27 de março de 2022 15:54',
 'R$ 34.767,00']

In [ ]:
# Criando funcao lista dado
def lista_dado(dados):
    for i, dado in enumerate(dados):
        dados[i] = dado[1]
    return dados

In [ ]:
# Buscando título
regex_titulo = re.compile('(td class="noborder"><p>)(.*)(</p>)')
titulo = regex_titulo.findall(str(table))
titulo = lista_dado(titulo)
titulo

['Mês de referência:',
 'Código Fipe:',
 'Marca:',
 'Modelo:',
 'Ano Modelo:',
 'Autenticação',
 'Data da consulta',
 'Preço Médio']

In [ ]:
# Limpar pesquisa e fechar navegador
esperar()
buscar = driver.find_element_by_id(id_botao_limpar)
buscar.click()

### Tarefa concluída, porém...
Concluímos nosso código, porém como podemos buscar um grupo maior de informações?

In [ ]:
# Buscando informações da caixa de seleção
referencia_ano = driver.find_element_by_id(id_tabela_referencia)
referencia_ano.click()

In [ ]:
# Extrai todos os dados da tabela de seleção
html_referencia = referencia_ano.get_attribute('innerHTML')

In [ ]:
re_anos = re.compile(r'(" style="">)(.*?)( </li>)')
lista_periodos = re_anos.findall(str(html_referencia))
lista_periodos

[('" style="">', 'março/2022', ' </li>'),
 ('" style="">', 'fevereiro/2022', ' </li>'),
 ('" style="">', 'janeiro/2022', ' </li>'),
 ('" style="">', 'dezembro/2021', ' </li>'),
 ('" style="">', 'novembro/2021', ' </li>'),
 ('" style="">', 'outubro/2021', ' </li>'),
 ('" style="">', 'setembro/2021', ' </li>'),
 ('" style="">', 'agosto/2021', ' </li>'),
 ('" style="">', 'julho/2021', ' </li>'),
 ('" style="">', 'junho/2021', ' </li>'),
 ('" style="">', 'maio/2021', ' </li>'),
 ('" style="">', 'abril/2021', ' </li>'),
 ('" style="">', 'março/2021', ' </li>'),
 ('" style="">', 'fevereiro/2021', ' </li>'),
 ('" style="">', 'janeiro/2021', ' </li>'),
 ('" style="">', 'dezembro/2020', ' </li>'),
 ('" style="">', 'novembro/2020', ' </li>'),
 ('" style="">', 'outubro/2020', ' </li>'),
 ('" style="">', 'setembro/2020', ' </li>'),
 ('" style="">', 'agosto/2020', ' </li>'),
 ('" style="">', 'julho/2020', ' </li>'),
 ('" style="">', 'junho/2020', ' </li>'),
 ('" style="">', 'maio/2020', ' </li>'),


In [ ]:
# Criando um dicionário de períodos
periodos = []
for periodo in lista_periodos:
    periodo = periodo[1]
    mes, ano = periodo.split('/')
    periodos.append({'periodo' : periodo, 'mes' : mes, 'ano' : ano})
periodos[0]

{'periodo': 'março/2022', 'mes': 'março', 'ano': '2022'}

In [ ]:
# Criando função de click
def click_item_id(id_item):
    item = driver.find_element_by_id(id_item)
    item.click()
    item.is_selecteds = False

In [ ]:
# Juntando tudo
lista_table = []
for periodo in periodos:
    if int(periodo['ano']) >= 2020:
        periodo_referencia = periodo['periodo']
        print(periodo_referencia)
        esperar()
        click_item_id(id_tabela_referencia)
        esperar()
        preenche_item(id_tabela_referencia, periodo_referencia)
        esperar()
        preenche_item(id_marca_carro, marca_veiculo)
        esperar()
        preenche_item(id_modelo_carro, modelo_veiculo)
        esperar()
        preenche_item(id_ano_carro, ano_veiculo)
        esperar()
        click_item_id(id_botao_pesquisa)
        esperar()
        table = driver.find_element_by_id(id_resultado_filtros)
        html = table.get_attribute('innerHTML')
        parser = BeautifulSoup(html, 'html.parser')
        table = parser.select_one('tbody')
        dados = regex_dados.findall(str(table))
        dados = lista_dado(dados)
        lista_table.append(dados)
        click_item_id(id_botao_limpar)

março/2022
fevereiro/2022
janeiro/2022
dezembro/2021
novembro/2021
outubro/2021
setembro/2021
agosto/2021
julho/2021
junho/2021
maio/2021
abril/2021
março/2021
fevereiro/2021
janeiro/2021
dezembro/2020
novembro/2020
outubro/2020
setembro/2020
agosto/2020
julho/2020
junho/2020
maio/2020
abril/2020
março/2020
fevereiro/2020
janeiro/2020


In [ ]:
regex_titulo = re.compile('(<td class="noborder"><p>)(.*?)(</p>)')
titulo = regex_titulo.findall(str(table))
titulo = lista_dado(titulo)
titulo

['Mês de referência:',
 'Código Fipe:',
 'Marca:',
 'Modelo:',
 'Ano Modelo:',
 'Autenticação',
 'Data da consulta',
 'Preço Médio']

In [ ]:
# Lendo dados
import pandas as pd

df = pd.DataFrame(lista_table, columns = titulo)

In [ ]:
df

,Mês de referência:,Código Fipe:,Marca:,Modelo:,Ano Modelo:,Autenticação,Data da consulta,Preço Médio
0,março de 2022,001314-5,Fiat,Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p,2013 Gasolina,rkmpgz8dbvnc,"domingo, 27 de março de 2022 15:38","R$ 34.994,00"
1,fevereiro de 2022,001314-5,Fiat,Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p,2013 Gasolina,rgy5qt8drlnc,"domingo, 27 de março de 2022 15:38","R$ 34.767,00"
2,janeiro de 2022,001314-5,Fiat,Palio ATTRA./ITÁLIA 1.4 EVO F.Flex 8V 5p,2013 Gasolina,rdz3r1ldpvnc,"domingo, 27 de março de 2022 15:39","R$ 34.599,00"


In [ ]:
driver.quit()

In [ ]:
df.to_csv('dados_palio_2013.csv', sep = ';', encoding = 'latin-1')

In [ ]:
periodos

[{'periodo': 'março/2022', 'mes': 'março', 'ano': '2022'},
 {'periodo': 'fevereiro/2022', 'mes': 'fevereiro', 'ano': '2022'},
 {'periodo': 'janeiro/2022', 'mes': 'janeiro', 'ano': '2022'},
 {'periodo': 'dezembro/2021', 'mes': 'dezembro', 'ano': '2021'},
 {'periodo': 'novembro/2021', 'mes': 'novembro', 'ano': '2021'},
 {'periodo': 'outubro/2021', 'mes': 'outubro', 'ano': '2021'},
 {'periodo': 'setembro/2021', 'mes': 'setembro', 'ano': '2021'},
 {'periodo': 'agosto/2021', 'mes': 'agosto', 'ano': '2021'},
 {'periodo': 'julho/2021', 'mes': 'julho', 'ano': '2021'},
 {'periodo': 'junho/2021', 'mes': 'junho', 'ano': '2021'},
 {'periodo': 'maio/2021', 'mes': 'maio', 'ano': '2021'},
 {'periodo': 'abril/2021', 'mes': 'abril', 'ano': '2021'},
 {'periodo': 'março/2021', 'mes': 'março', 'ano': '2021'},
 {'periodo': 'fevereiro/2021', 'mes': 'fevereiro', 'ano': '2021'},
 {'periodo': 'janeiro/2021', 'mes': 'janeiro', 'ano': '2021'},
 {'periodo': 'dezembro/2020', 'mes': 'dezembro', 'ano': '2020'},
 {'p